(continuation from [Task 1](1-ingestion.ipynb))
#### Task 2. Analyze the raw data and describe it, in any way that is relevant and informative for the problem at hand.
Importing libraries and defining constants

In [ ]:
import sqlite3
DB_FILE = 'task-2.db'

Defining a general function to read from database

In [ ]:
def dbQuery(sql):
    conn = sqlite3.connect(DB_FILE)
    for row in conn.execute(sql):
        yield row
    conn.close()

### Topics
The corpus is distributed into 10 topics as follows

In [ ]:
sql = '''
SELECT 'topic-' || topicId, c, ROUND(c * 100.0 /total, 2) || '%' FROM (
 SELECT topicId, COUNT(*) AS c,
  (SELECT COUNT(*) FROM conversation_topic) AS total
 FROM conversation_topic
 GROUP BY topicId
)
ORDER BY c DESC
'''
[x for x in dbQuery(sql)]

Taking random conversation of each topic we can infer what are they about (**change the value to explore**)

In [ ]:
topicId = 8 # Change the value to explore

sql = '''
SELECT GROUP_CONCAT(Utterance, ' -')
FROM utterance 
WHERE cId IN (
 SELECT cId FROM conversation_topic WHERE topicId = {}
 ORDER BY RANDOM()
 LIMIT 2
)
GROUP BY cId
'''.format(topicId)
for x in dbQuery(sql): print('{}\n'.format(x[0]))

### Acts
There are 4 different [dialog acts](https://en.wikipedia.org/wiki/Dialog_act) distributed as follows

In [ ]:
sql = '''
SELECT 'act-' || actId, c, ROUND(c * 100.0 / total, 2) || '%'
FROM (
 SELECT actId, COUNT(*) AS c, (SELECT COUNT(*) FROM utterance) AS total
 FROM utterance_act
 GROUP BY actId
)
ORDER BY c DESC
'''
[x for x in dbQuery(sql)]

Theorically speaking dialog acts are conditioned by their previous act, so for example an initial act is very unlikely to be *act-4*

In [ ]:
sql = '''
SELECT 'act-' || actId, COUNT(*) AS c 
FROM utterance_act WHERE uId = 1
GROUP BY actId
ORDER BY c
'''
[x for x in dbQuery(sql)]

There are only a few cases with *act-4*. Let's look at them closely

In [ ]:
sql = '''
SELECT Utterance
FROM utterance_act AS a
 INNER JOIN utterance AS u ON (u.cId = a.cId AND u.uId = a.uId)
WHERE actId = 4 AND a.uId = 1
'''
[x[0] for x in dbQuery(sql)]

**Intuition**: *Conversations may ocurr in broader contexts not contained in these dialogs. Like a bus arriving the station or other acts (or omitted acts) between the participants.*

Now if *act-4* is highly dependent of previous acts, which are these inside the corpus? How likely are they to elicit an *act-4*?

In [ ]:
sql = '''
SELECT actId, COUNT(*) AS c
FROM utterance_act
WHERE (cId, uId) IN (
 SELECT cId, (uId - 1) AS uId
 FROM utterance_act
 WHERE uId > 1 AND actId = 4
)
GROUP BY actId
ORDER BY c DESC
'''
[x for x in dbQuery(sql)]

**Intuition**: *Preponderantly, act-3 (which presumably is a YES-NO-QUESTION) preceeds act-4, and therefore their ocurrance at the end of the conversation should be rather low* 

But moreover, are these question also identifiable by punctuation marks, or what other forms they have? Let's look at this:

In [ ]:
sql = '''
SELECT (CASE WHEN Utterance LIKE '%?%' THEN 'marked' ELSE 'unmarked' END) AS Class, COUNT(Utterance)
FROM utterance_act AS a
 INNER JOIN utterance AS u ON (u.cId = a.cId AND u.uId = a.uId)
WHERE actId = 3
GROUP BY Class
'''
[x for x in dbQuery(sql)]

There are surprisingly more *unmarked* utterances! Let's take a closer look to some samples

In [ ]:
sql = '''
SELECT Utterance
FROM utterance_act AS a
 INNER JOIN utterance AS u ON (u.cId = a.cId AND u.uId = a.uId)
WHERE actId = 3 AND Utterance NOT LIKE '%?%'
ORDER BY RANDOM()
LIMIT 10
'''
[x[0] for x in dbQuery(sql)]

**Intuition**: *Besides question marks, ther are other [discourse markers](https://en.wikipedia.org/wiki/Discourse_marker) that characterize act-3 like*:
* I (might) need...
* ...sure we can...
* ...in that case...
* Perhaps... 
* ...would be... if
* ...will need to...
* Don't...
* Set your...
* Let me...
* Let's get you...
* Tell me..

As we dive in more cases, we can identify some utterances use imperatives while others rather insinuate commands or questions via *indirect speech*. It might be worth thinking on unfolding this into subtypes that characterize this long continuum.

### Emotions
There are 6 distinct emotions and one *emotion-0* distributed as follows

In [ ]:
sql = '''
SELECT 'emotion-' || emotionId, c, ROUND(c * 100.0 / total, 2) || '%'
FROM (
 SELECT emotionId, COUNT(*) AS c, (SELECT COUNT(*) FROM utterance) AS total
 FROM utterance_emotion
 GROUP BY emotionId
)
ORDER BY c DESC
'''
[x for x in dbQuery(sql)]

Similarly to topics, we can explore random samples to grasp what are the emotions annotated in the corpus (**change the value to explore**)

In [ ]:
emotionId = 6 # Change the value to explore

sql = '''
SELECT Utterance FROM utterance 
WHERE (cId, uId) IN (
 SELECT cId, uId FROM utterance_emotion
 WHERE emotionId = {}
 ORDER BY RANDOM()
 LIMIT 10
)
'''.format(emotionId)
[x[0] for x in dbQuery(sql)]

So we can state this is our set of emotions:
0. (void)
1. Disagreeement
2. Disappointment 
3. Fear
4. Agreement
5. Apology
6. Excitement

Let's start explorin apologies(5). Can we assume they're **only** followed by utterances of agreement(4) or disagreement(1)?

In [ ]:
sql = '''
SELECT emotionId, COUNT(*) AS c
FROM utterance_emotion
WHERE (cId, uId) IN (
 SELECT cId, (uId + 1) AS uId
 FROM utterance_emotion
 WHERE emotionId = 5
)
GROUP BY emotionId
HAVING emotionId != 0 -- discarding emotion-0
ORDER BY c DESC
'''
[x for x in dbQuery(sql)]

Apologies are mostly followed by any kind of emotion except disappointment(2) and fear(3). Also apologies are often followed by apologies.

Now talking about excitement(6): Similarly to questions, are there any discourse markers for that we can identify (besides exclamation marks)?

In [ ]:
sql = '''
SELECT Utterance
FROM utterance_emotion AS e
 INNER JOIN utterance AS u ON (u.cId = e.cId AND u.uId = e.uId)
WHERE emotionId = 6 AND Utterance NOT LIKE '%!%'
ORDER BY RANDOM()
LIMIT 10
'''
[x[0] for x in dbQuery(sql)]

**Intuition**: *As with YES-NO-QUESTIONs, there are plenty of markers (apart from exlamation marks) that characterize emotion-6 like*:
* Oh
* Oh, my
* Oh, really
* amazing
* gorgeous
* Look at this
* ...crazy?
* ...kidding?
* Really?
* Pardon?
* ...serious?
* Seriously?
* So?
* What?
* Why is so...?

### Mashups
Further analysis can be made combining features. As noted in the last result, question marks are not only used to inquiry (act-3) but also as rethoric exclamations (emotion-6). Let's ponderate the ocurrence of this mark among these two

In [ ]:
sql = '''
-- questions
SELECT 'act-' || actId, COUNT(*) AS c
FROM utterance AS u
 INNER JOIN utterance_act AS a ON (a.cId = u.cId AND a.uId = u.uId)
 INNER JOIN utterance_emotion AS e ON (e.cId = u.cId AND e.uId = u.uId)
WHERE Utterance LIKE '%?%' AND actId = 3 AND emotionId != 6
GROUP BY actId
-- excitement
UNION
SELECT 'emotion-' || emotionId, COUNT(*) AS c
FROM utterance AS u
 INNER JOIN utterance_act AS a ON (a.cId = u.cId AND a.uId = u.uId)
 INNER JOIN utterance_emotion AS e ON (e.cId = u.cId AND e.uId = u.uId)
WHERE Utterance LIKE '%?%' AND actId != 3 AND emotionId = 6
GROUP BY emotionId
-- both
UNION
SELECT 'both', COUNT(*) AS c
FROM utterance AS u
 INNER JOIN utterance_act AS a ON (a.cId = u.cId AND a.uId = u.uId)
 INNER JOIN utterance_emotion AS e ON (e.cId = u.cId AND e.uId = u.uId)
WHERE Utterance LIKE '%?%' AND actId = 3 AND emotionId = 6
ORDER BY c DESC
'''
[x for x in dbQuery(sql)]

Although question marks are very likely to denote a question, they are also used fairly on the context of exclamation or suprise (but very rarely they combine the two).
### Further Analysis
As you may concluded, there are infinite ways to explore this data and discover patterns for the study of dual conversations. Feel free to run your own queries here:

In [ ]:
sql = '''
SELECT 'your query here'
'''
[x for x in dbQuery(sql)]